## <center> Gapminder Emulator with Altair </center> 

### Module and Data Inputs

In [2]:
import pandas as pd
import altair as alt
from vega_datasets import data

**Vega datasets includes gapminder data**

In [3]:
gapminder = pd.read_excel("gapminder.xlsx", sheet_name="gapminder")

In [4]:
gapminder.head()

,Continent,Country,ISOCode,Year,Population,LifeExp,GDPPC
0,Asia,Afghanistan,4,1952,8425333,28.801,779.445314
1,Asia,Afghanistan,4,1957,9240934,30.332,820.853030
2,Asia,Afghanistan,4,1962,10267083,31.997,853.100710
3,Asia,Afghanistan,4,1967,11537966,34.020,836.197138
4,Asia,Afghanistan,4,1972,13079460,36.088,739.981106


### Creade year selection slider to use on graphs

In [5]:
year_input = alt.binding_range(min=gapminder["Year"].min(), max=gapminder["Year"].max(), step=5, name="Year")
year_slider = alt.selection_single(name="Year", fields= ["Year"], bind=year_input, init={"Year": 1952})

### GDP Per Capita vs Life Expectancy Bubble Chart (Trendalizer)

In [6]:
bubble_chart = alt.Chart(gapminder).mark_point(filled=True, opacity=0.7).encode(
    x=alt.X("GDPPC:Q", title="GDPPC", scale=alt.Scale(domain=(0, 50000))),
    y=alt.Y("LifeExp:Q", title="Life Expectancy", scale=alt.Scale(domain=(0, 90))), 
    color=alt.Color("Continent:N"),
    size="Population:Q",
    tooltip=[alt.Tooltip("Country:N"), 
             alt.Tooltip("Year:Q"), 
             alt.Tooltip("GDPPC:Q", title = "GDP Per Capita", format="$,.0f"), 
             alt.Tooltip("LifeExp:Q", title="Life Expectancy", format=".0f")]
)

watermark = alt.Chart(gapminder).mark_text(
    align='center', dx=0, dy=18, fontSize=64, fontWeight=200, color="lightgrey"
).encode(text="Year:Q")

bubble_graph = bubble_chart + watermark

In [7]:
bubble_graph.add_selection(year_slider).transform_filter(year_slider).properties(width=600, height=300)

alt.LayerChart(...)

### Life Expectancy Histogram and Distribution

In [8]:
LifeExpVal = alt.Chart(gapminder).mark_bar().encode(
    x=alt.X("LifeExp:Q", bin=alt.Bin(extent=[25, 85], step=5), title="Life Expectancy"),
    y=alt.Y("count()", scale=alt.Scale(domain=(0, 40)), title="Count"),
    color=alt.Color("Continent:N")
).properties(width=400, height=300)

In [9]:
LifeExpPct = alt.Chart(gapminder).transform_joinaggregate(
    Total ='count(*)'
).transform_calculate(
    Percent = "1 / datum.Total"
).mark_bar().encode(
    x=alt.X("LifeExp:Q", bin=alt.Bin(extent=[25, 85], step=5), title="Life Expectancy"),
    y=alt.Y("sum(Percent):Q", axis=alt.Axis(format='.0%'), scale=alt.Scale(domain=(0, 0.3)), title="Percent"),
    color=alt.Color("Continent:N")
).properties(width=400, height=300)

In [10]:
(LifeExpVal | LifeExpPct).add_selection(year_slider).transform_filter(year_slider)

alt.HConcatChart(...)

### GDP Per Capita Histogram and Distribution

In [11]:
GDPPC = alt.Chart(gapminder).mark_bar().encode(
    x=alt.X("GDPPC:Q", bin=alt.Bin(extent=[0, 60000], step=5000), title="GDP Per Capita"),
    y=alt.Y("count()", scale=alt.Scale(domain=(0, 120)), title="Count"),
    color=alt.Color("Continent:N")
).properties(width=400, height=300)

In [12]:
GDPPCPct = alt.Chart(gapminder).transform_joinaggregate(
    Total ='count(*)'
).transform_calculate(
    Percent = "1 / datum.Total"
).mark_bar().encode(
    x=alt.X("GDPPC:Q", bin=alt.Bin(extent=[0, 60000], step=5000), title="GDPPC"),
    y=alt.Y("sum(Percent):Q", axis=alt.Axis(format='.0%'), scale=alt.Scale(domain=(0, .85)), title="Percent"),
    color=alt.Color("Continent:N")
).properties(width=400, height=300)

In [13]:
(GDPPC | GDPPCPct).add_selection(year_slider).transform_filter(year_slider)

alt.HConcatChart(...)

### Set up background map for Chloropleth Wolrd Map

In [14]:
# Source of world land data
countries = alt.topo_feature(data.world_110m.url, "countries")
# Background map to "attach" data to
background = alt.Chart(countries).mark_geoshape(fill="lightgrey", stroke="white").project(
    type="equirectangular").properties(width=800, height=350)

### Life Expectancy Chloropleth World Map

In [16]:
LifeExpMap = alt.Chart(countries).mark_geoshape(stroke="white").encode(
    color=alt.Color("LifeExp:Q", scale=alt.Scale(scheme='blues')),
    tooltip=[alt.Tooltip("Country:N"), 
             alt.Tooltip("Year:Q"), 
             alt.Tooltip("LifeExp:Q", title="Life Expectancy", format=".0f")]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(gapminder, "ISOCode", ["Country", "Year", "LifeExp"])
).project(type="equirectangular").properties(width=800, height=350)

In [17]:
alt.layer(background, LifeExpMap)

alt.LayerChart(...)

### GDP Per Capita Chloropleth World Map

In [18]:
GDPPCMap = alt.Chart(countries).mark_geoshape(stroke="white").encode(
    color=alt.Color("GDPPC:Q", scale=alt.Scale(scheme='greens')),
    tooltip=[alt.Tooltip("Country:N"), 
             alt.Tooltip("Year:Q"), 
             alt.Tooltip("GDPPC:Q", title = "GDP Per Capita", format="$,.0f")]
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(gapminder, "ISOCode", ["Country", "Year", "GDPPC"])
).project(type="equirectangular").properties(width=800, height=350)

In [19]:
alt.layer(background, GDPPCMap)

alt.LayerChart(...)